# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
# def get_content(url):
#     with requests.Session() as req:
#         req.headers.update(headers)
#         r = req.get(url).content
#     return r

# url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
# s = get_content(url)
# df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Если не работает загрузка on-line
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", engine='python',  sep=';', encoding='utf-8-sig')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [76]:
# Ваше решение
target_df = df.copy()

In [77]:
# заполним значение б/цвета 0
target_df['Результат анализа'] = target_df['Результат анализа'].apply(lambda x: 0 if x == 'б/цвета' else x)

assert(target_df[target_df['Результат анализа'] == 'б/цвета'].shape[0] == 0)

In [78]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
Показатель           14 non-null object
Единица измерений    14 non-null object
Результат анализа    14 non-null object
Норматив             14 non-null object
dtypes: object(4)
memory usage: 528.0+ bytes


In [79]:
# колонку Результат анализа приведем в тип float
target_df['Результат анализа'] = target_df['Результат анализа'].astype(float)

In [80]:
# зададим левую границу норматива
target_df['Норматив_старт'] = target_df['Норматив'].str.extract(r'в\sпределах\s(\d*,?\d+)-')
target_df['Норматив_старт'].fillna(0, inplace=True) 
target_df

,Показатель,Единица измерений,Результат анализа,Норматив,Норматив_старт
0,pH,единицы pH,8.400,в пределах 6-9,6
1,Запах,баллы,1.000,не более 2-3,0
2,Цветность,градусы,0.000,не более 30,0
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,7
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",0
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,0
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",0
8,Хлориды (Cl),мг/дм3,200.000,не более 350,0
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,0


In [81]:
# зададим правую границу норматива
def set_end_normativ(value):
    pattern = re.compile(r'(?<=в\sпределах\s)\d+-(\d*,?\d+)|(?<=не\sболее\s)\d-(\d*,?\d+)|(?<=не\sболее\s)(\d*,?\d+)')
    matches = pattern.findall(value)
    
    if len(matches) == 0:
        return None
    
    for match in matches[0]:
        if match != '':
            return match

target_df['Норматив_конец'] = target_df['Норматив'].apply(set_end_normativ)

target_df

,Показатель,Единица измерений,Результат анализа,Норматив,Норматив_старт,Норматив_конец
0,pH,единицы pH,8.400,в пределах 6-9,6,9
1,Запах,баллы,1.000,не более 2-3,0,3
2,Цветность,градусы,0.000,не более 30,0,30
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,7,10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",0,"1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0,"3,3"
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,0,45
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",0,"3,5"
8,Хлориды (Cl),мг/дм3,200.000,не более 350,0,350
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,0,500


In [82]:
# заменим запятые на точки, для перевода в тип float
target_df['Норматив_старт'] = target_df['Норматив_старт'].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
target_df['Норматив_конец'] = target_df['Норматив_конец'].apply(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
target_df

,Показатель,Единица измерений,Результат анализа,Норматив,Норматив_старт,Норматив_конец
0,pH,единицы pH,8.400,в пределах 6-9,6,9
1,Запах,баллы,1.000,не более 2-3,0,3
2,Цветность,градусы,0.000,не более 30,0,30
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,7,10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",0,1.5
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0,3.3
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,0,45
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",0,3.5
8,Хлориды (Cl),мг/дм3,200.000,не более 350,0,350
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,0,500


In [83]:
# приведем Норматив_страт и Норматив_конец к формату float
target_df['Норматив_старт'] = target_df['Норматив_старт'].astype(float)
target_df['Норматив_конец'] = target_df['Норматив_конец'].astype(float)
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
Показатель           14 non-null object
Единица измерений    14 non-null object
Результат анализа    14 non-null float64
Норматив             14 non-null object
Норматив_старт       14 non-null float64
Норматив_конец       14 non-null float64
dtypes: float64(3), object(3)
memory usage: 752.0+ bytes


In [84]:
# заполним вывод по нормативу
target_df['Результат_по_нормативу'] =  (target_df['Норматив_старт'] <= target_df['Результат анализа']) & (target_df['Результат анализа'] <= target_df['Норматив_конец'])
target_df

,Показатель,Единица измерений,Результат анализа,Норматив,Норматив_старт,Норматив_конец,Результат_по_нормативу
0,pH,единицы pH,8.400,в пределах 6-9,6.0,9.0,True
1,Запах,баллы,1.000,не более 2-3,0.0,3.0,True
2,Цветность,градусы,0.000,не более 30,0.0,30.0,True
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,7.0,10.0,True
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",0.0,1.5,True
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0.0,3.3,True
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,0.0,45.0,True
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",0.0,3.5,True
8,Хлориды (Cl),мг/дм3,200.000,не более 350,0.0,350.0,True
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,0.0,500.0,True


In [86]:
# сделаем колнку Показатель индексной
result_df = target_df.set_index('Показатель')
result_df

,Единица измерений,Результат анализа,Норматив,Норматив_старт,Норматив_конец,Результат_по_нормативу
Показатель,,,,,,
pH,единицы pH,8.400,в пределах 6-9,6.0,9.0,True
Запах,баллы,1.000,не более 2-3,0.0,3.0,True
Цветность,градусы,0.000,не более 30,0.0,30.0,True
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,7.0,10.0,True
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",0.0,1.5,True
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",0.0,3.3,True
Нитраты (по NO3),мг/дм3,24.000,не более 45,0.0,45.0,True
Фосфаты (P),мг/дм3,0.360,"не более 3,5",0.0,3.5,True
Хлориды (Cl),мг/дм3,200.000,не более 350,0.0,350.0,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [133]:
# Ваше решение
import random
box = ['orange'] * 5 + ['apple'] * 4

n = 500000
A = pd.DataFrame({"A":list(map(lambda a: ''.join(random.sample(box, k=choose)), range(n)))})

p_orange = len(A[A.A=='orange' * 3])/A.shape[0]
p_orange

0.119316

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
# Ваше решение

In [182]:
s_cases = 0
n = 1000000

for _ in range(n):
    details = [0] * 7 + [1] * 3
    case1 = random.choice(details)
    details.remove(case1)
    case2 = random.choice(details)
    
    if case1 == 0:
        continue
    
    if case2 == 0:
        s_cases += 1
        
print(s_cases / n)

0.233198